In [ ]:
import time

BUILDPLATFORM = 'mps2-an386'
BUILDPLATFORM = 'stm32f4discovery'
BUILDPLATFORM = 'cw308t-stm32f3'
BUILDPLATFORM = 'cw308t-stm32f415'
lvl = "l1"
m4f = False # implementation switch, either use m4f or mupq implementation
# for m4f only levels 1 and 3 are implemented

ff = f"{'' if m4f else 'mupq_'}crypto_kem_bike{lvl}_{'m4f' if m4f else 'opt'}_kat"

%run "Setup_Disco_or_CWLITE.ipynb"

In [ ]:
%run "Build_Firmware.ipynb"

In [ ]:
start = time.time()
%run "Flash_Disco_CWLITE.ipynb"
end = time.time()
print(end-start)

In [ ]:
# For physical targets

import kat_bike as kat
start = time.time()
# actually run KAT
# read the first N runs before comparing to the KAT
N = 30

ref_kat = kat.read_rsp(lvl)

reboot_flush()
target_kat = kat.read_target(target, lvl, N)

check = True
for k in target_kat.keys():
	if not ref_kat[k] == target_kat[k]:
		print(f"in round {ref_kat[k].count} reference and target do not match")
		check = False

print(f"{len(target_kat)} of {N} KAT iterations were successfully transmitted.")
if check:
	print("KAT was successful!\nReference implementation and target generate same keys!")
else:
    print("Error!\nReference Implementation and target do not match.")
end = time.time()
print(end-start)

In [ ]:
# For mps2-an386 target

import kat_bike as kat
import subprocess as sp
from os import remove

sp.call(f"qemu-system-arm -M mps2-an386 -nographic -semihosting -kernel ../pqm4/elf/{ff}.elf > tmp", shell=True)

rsp = kat.read_rsp(lvl)
f=open("tmp")
mps2 = kat.read_target(f,lvl)
f.close()
remove("tmp")

check = True
for k in mps2.keys():
	if not rsp[k] == mps2[k]:
		print(f"in round {rsp[k].count} reference and target do not match")
		check = False

if check:
	print("KAT was successful!\nReference implementation and target generate same keys!")